<a href="https://colab.research.google.com/github/Tejaswidarsi/ECC/blob/main/Lab_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1)Write a program to perform a key exchange using ECC.

In [4]:
from cryptography.hazmat.primitives.asymmetric import ec

# Use the same curve for both parties (e.g., SECP256R1)
curve = ec.SECP256R1()

# Generate private keys for two parties
private_key_A = ec.generate_private_key(curve)
private_key_B = ec.generate_private_key(curve)

# Generate public keys
public_key_A = private_key_A.public_key()
public_key_B = private_key_B.public_key()

# Exchange keys and derive shared secret
shared_secret_A = private_key_A.exchange(ec.ECDH(), public_key_B)
shared_secret_B = private_key_B.exchange(ec.ECDH(), public_key_A)

# Ensure both shared secrets are the same
assert shared_secret_A == shared_secret_B, "Key exchange failed!"

print("Shared Secret:", shared_secret_A.hex())


Shared Secret: 8cf7b5ac948c7a26fc15a0c55a5aa80c91fbe1d1c6e52951358c348ef6b69181


2) Write a program to encrypt and decrypt the message “Hell0 SRM AP” using Elliptic Curve
Cryptography and AES.

In [5]:
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

# Message to encrypt
message = b"Hell0 SRM AP"

# Generate ECC key pair
private_key = ec.generate_private_key(ec.SECP256R1())
public_key = private_key.public_key()

# Encrypt function
def encrypt_message(public_key, message):
    ephemeral_key = ec.generate_private_key(ec.SECP256R1())
    shared_secret = ephemeral_key.exchange(ec.ECDH(), public_key)

    # Derive AES key using HKDF
    aes_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"encryption",
    ).derive(shared_secret)

    iv = os.urandom(16)
    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
    encryptor = cipher.encryptor()

    padded_message = message + b' ' * (16 - len(message) % 16)  # Padding
    ciphertext = encryptor.update(padded_message) + encryptor.finalize()

    return ephemeral_key.public_key(), iv, ciphertext

# Decrypt function
def decrypt_message(private_key, ephemeral_public_key, iv, ciphertext):
    shared_secret = private_key.exchange(ec.ECDH(), ephemeral_public_key)

    aes_key = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b"encryption",
    ).derive(shared_secret)

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv))
    decryptor = cipher.decryptor()

    decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()
    return decrypted_padded.strip()

# Encrypt message
ephemeral_pub, iv, ciphertext = encrypt_message(public_key, message)
print("Ciphertext:", ciphertext.hex())

# Decrypt message
decrypted_message = decrypt_message(private_key, ephemeral_pub, iv, ciphertext)
print("Decrypted Message:", decrypted_message.decode())


Ciphertext: e5524228a3ffbe4e134baed3c222858b
Decrypted Message: Hell0 SRM AP
